## ExaGO Demo 2

In this demo, we will use ExaGO's python wrappers to

1. Run an AC optimal power flow (ACOPF) with load loss activated
2. Run security constrained ACOPF
3. Run security constrained ACOPF on multiple processors
4. Run stochastic security constrained ACOPF
5. Run stochastic security constrained ACOPF on multiple processors

In [ ]:
import os
import exago
## Initialize ExaGO
exago.initialize("exago_examples")

#### 1. AC optimal power flow with load loss enabled

In [ ]:
input_file='/Users/abhy245/software/ExaGO/datafiles/case9/case9mod_loadloss.m'

# Create ExaGO object
opf = exago.OPFLOW()

# Read Data
opf.read_mat_power_data(input_file)

# Enable load loss
opf.set_has_loadloss(True)

# Set load loss penalty
opf.set_loadloss_penalty(1000)

# Solve
opf.solve()

# Print solution
opf.print_solution()

# Save solution
opf.save_solution(exago.OutputFormat.JSON, 'case9sol')

del opf

import json

fp = open('case9sol.json')
dict = json.load(fp)

dict['summary']


#### 2. Run security-constrained ACOPF

In [ ]:
import exago
import os

# Create security-constrained ACOPF (SCOPF) object
scopf = exago.SCOPFLOW()

netfile='/Users/abhy245/software/ExaGO/datafiles/case9/case9mod.m'
ctgcfile='/Users/abhy245/software/ExaGO/datafiles/case9/case9.cont'

! echo Contingency file
! cat $ctgcfile

# Set network data
scopf.set_network_data(netfile)

# Set contingency data file and input format
scopf.set_contingency_data(ctgcfile,exago.ContingencyFileInputFormat.NATIVE)

# Set number of contingencies, we select all (-1)
scopf.set_num_contingencies(-1)

# Set solver
# Note: EMPAR solver runs each ACOPF independently.
# To solve true SCOPF, use solver HIOP
scopf.set_solver("EMPAR")

# Solve SCOPF
scopf.solve()

# Print base-case solution
scopf.print_solution(0)

# Save all contingency solutions to directory case9scopf, use MATPOWER formatted files
scopf.save_solution_all(exago.OutputFormat.MATPOWER, 'case9scopf')

# delete scopf object
del scopf


#### 3. Run Security-constrained Optimal Power Flow on multiple processors

In [ ]:

! cat scopflow-example.py


#### Below code does not work from within Jupyter notebook. Need to execute it from command line

In [ ]:
! mpiexec -n 4 python3 scopflow-example.py

#### 4. Run stochastic security constrained ACOPF

In [ ]:
import exago
import os

# Input files
netfile='/Users/abhy245/software/ExaGO/datafiles/case9/case9mod.m'
ctgcfile='/Users/abhy245/software/ExaGO/datafiles/case9/case9.cont'
scenfile='/Users/abhy245/software/ExaGO/datafiles/case9/10_scenarios_9bus.csv'

! echo Scenario File
! cat $scenfile
! echo Contingency File
! cat $ctgcfile

# Create Stochastic optimal power flow (SOPF) object
sopf = exago.SOPFLOW()

# Set network data
sopf.set_network_data(netfile)

# Set contingency data file and input format
sopf.set_contingency_data(ctgcfile,exago.ContingencyFileInputFormat.NATIVE)

# Set scenario data
sopf.set_scenario_data(scenfile,exago.ScenarioFileInputFormat.NATIVE_SINGLEPERIOD,
    exago.ScenarioUncertaintyType.WIND)

# Set number of scenarios, we select 2
sopf.set_num_scenarios(2)

# Set number of contingencies, we select 2
sopf.set_num_contingencies(2)

# Enable multi-contingency
sopf.enable_multi_contingency(True)

# Set solver
sopf.set_solver("EMPAR")

# Solve SCOPF
sopf.solve()

# Print base-case solution
sopf.print_solution(0)

# Save all contingency solutions to directory case9scopf, use MATPOWER formatted files
sopf.save_solution_all(exago.OutputFormat.MATPOWER, 'case9sopf')

# delete object
del sopf

In [ ]:
# Close ExaGO
exago.finalize()

#### 5. Run stochastic security-constrained ACOPF on multiple processors

In [ ]:
! cat sopflow-example.py

#### Below code does not work from within Jupyter notebook. Need to execute it from command line

In [ ]:
! mpiexec -n 4 python3 sopflow-example.py